In [1]:
import pandas as pd
import numpy as np
from unidecode import unidecode
from glob import glob
from tqdm.notebook import tqdm

# Uso: 

Para usar esse código é nessário que baixe alguns dados disponíveis no [site](https://opendatasus.saude.gov.br/gl/dataset/casos-nacionais) e coloque na mesma pasta desse arquivo.
Também será necessário que não tenha uma pasta com o nome data_city no diretório deste arquivo, pois irá ser apagada. 

In [2]:
def strres(s: str) -> str:
    return unidecode(s.lower())

In [3]:
df = pd.Series(dtype='float64')
all_files = glob('./' + 'dados-*.csv')

for file in all_files:
    df_tmp = pd.read_csv(file, encoding='latin-1', delimiter=';')
    df = pd.concat([df, df_tmp], ignore_index=True)

df = df.drop([
        'dataNotificacao',
        'dataTeste',
        'tipoTeste',
        'dataEncerramento',
        'evolucaoCaso',
        'classificacaoFinal',
        'estadoNotificacaoIBGE',
        'cbo',
        'municipioNotificacaoIBGE',
        'estadoIBGE',
        'estadoTeste',
        'paisOrigem',
        'origem',
        'estadoNotificacao',
        'municipioNotificacao',
        'excluido', 
        'validado',
        'dataInicioSintomas',
        'profissionalSaude',
        'dataNascimento',
        'estado',
        'municipio',
        0,
        'ÿid'
        ], axis=1)

df.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,condicoes,idade,municipioIBGE,resultadoTeste,sexo,sintomas
0,NaN,32.0,5208707.0,NaN,Masculino,"Outros, Dor de Cabeça, Febre"
1,NaN,21.0,5208004.0,Positivo,Masculino,Assintomático
2,NaN,39.0,5208608.0,Negativo,Masculino,"Tosse, Coriza"
3,NaN,20.0,5217401.0,Negativo,Masculino,"Dor de Cabeça, Tosse, Outros"
4,NaN,31.0,5208707.0,Positivo,Masculino,"Assintomático, Febre, Tosse, Dor de Cabeça, Di..."


In [4]:
# Transformando coluna sintomas em várias

sintomas = list([
    'paladar', 
    'fadiga',  
    'olfato',
    'garganta',
    'respirar', 
    'febre',    
    'tosse',
    'diarreia',
    'coriza',
    'dispineia',
])

# Criando novas colunas para sintomas

df[sintomas] = 0
for sint in tqdm(sintomas):
    li = []
    for s in df['sintomas']:
        if strres(str(s)).find(sint) != -1:
            li.append(1)
        else:
            li.append(0)
    df[sint] = li

df = df.drop(['sintomas'], axis=1)
print(set(df['resultadoTeste']))
print(set(df['sexo']))
df.head()

  0%|          | 0/10 [00:00<?, ?it/s]

{nan, 'undefined', 'Negativo', 'Inconclusivo ou Indeterminado', 'Positivo'}
{nan, 'Masculino', 'Indefinido', 'Feminino'}


,condicoes,idade,municipioIBGE,resultadoTeste,sexo,paladar,fadiga,olfato,garganta,respirar,febre,tosse,diarreia,coriza,dispineia
0,NaN,32.0,5208707.0,NaN,Masculino,0,0,0,0,0,1,0,0,0,0
1,NaN,21.0,5208004.0,Positivo,Masculino,0,0,0,0,0,0,0,0,0,0
2,NaN,39.0,5208608.0,Negativo,Masculino,0,0,0,0,0,0,1,0,1,0
3,NaN,20.0,5217401.0,Negativo,Masculino,0,0,0,0,0,0,1,0,0,0
4,NaN,31.0,5208707.0,Positivo,Masculino,0,0,0,0,0,1,1,0,1,0


In [5]:
# Convertando coluna condicoes para valores numéricos

li = []
for cond in df['condicoes']:
    if str(cond).lower() != 'nan':
        li.append(1)
    else:
        li.append(0)
df['condicoes'] = li

print(set(df['condicoes']))
df.head()

{0, 1}


,condicoes,idade,municipioIBGE,resultadoTeste,sexo,paladar,fadiga,olfato,garganta,respirar,febre,tosse,diarreia,coriza,dispineia
0,0,32.0,5208707.0,NaN,Masculino,0,0,0,0,0,1,0,0,0,0
1,0,21.0,5208004.0,Positivo,Masculino,0,0,0,0,0,0,0,0,0,0
2,0,39.0,5208608.0,Negativo,Masculino,0,0,0,0,0,0,1,0,1,0
3,0,20.0,5217401.0,Negativo,Masculino,0,0,0,0,0,0,1,0,0,0
4,0,31.0,5208707.0,Positivo,Masculino,0,0,0,0,0,1,1,0,1,0


In [6]:
# Convertendo coluna de resultado de teste em dados numéricos.

li = []
for res in df['resultadoTeste']:
    if str(res) == 'Positivo':
        li.append(1)
    elif str(res).lower() == 'nan':
        li.append('nan')
    else:
        li.append(0)
df['resultadoTeste'] = li

print(set(df['resultadoTeste']))
df.head()

{0, 1, 'nan'}


,condicoes,idade,municipioIBGE,resultadoTeste,sexo,paladar,fadiga,olfato,garganta,respirar,febre,tosse,diarreia,coriza,dispineia
0,0,32.0,5208707.0,nan,Masculino,0,0,0,0,0,1,0,0,0,0
1,0,21.0,5208004.0,1,Masculino,0,0,0,0,0,0,0,0,0,0
2,0,39.0,5208608.0,0,Masculino,0,0,0,0,0,0,1,0,1,0
3,0,20.0,5217401.0,0,Masculino,0,0,0,0,0,0,1,0,0,0
4,0,31.0,5208707.0,1,Masculino,0,0,0,0,0,1,1,0,1,0


In [7]:
# Convertendo campo sexo em dados numéricos

li = []
for sexo in df['sexo']:
    if sexo == 'Masculino':
        li.append(0)
    elif sexo == 'Feminino':
        li.append(1)
    else:
        li.append('nan')
df['sexo'] = li

print(set(df['sexo']))
df.head()

{0, 1, 'nan'}


,condicoes,idade,municipioIBGE,resultadoTeste,sexo,paladar,fadiga,olfato,garganta,respirar,febre,tosse,diarreia,coriza,dispineia
0,0,32.0,5208707.0,nan,0,0,0,0,0,0,1,0,0,0,0
1,0,21.0,5208004.0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,39.0,5208608.0,0,0,0,0,0,0,0,0,1,0,1,0
3,0,20.0,5217401.0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,31.0,5208707.0,1,0,0,0,0,0,0,1,1,0,1,0


In [8]:
# Removendo linhas incorretas

li = []

for i in tqdm(range(len(df.iloc[:,:].values))):
    line = df.iloc[i,:].values
    catch = sum(line[5:]) == 0
    for dado in line:
        if str(dado).lower() == 'nan':
            catch = True
            break
    if catch:
        li.append(i)
df = df.drop(li)
df.head()

  0%|          | 0/2608107 [00:00<?, ?it/s]

,condicoes,idade,municipioIBGE,resultadoTeste,sexo,paladar,fadiga,olfato,garganta,respirar,febre,tosse,diarreia,coriza,dispineia
2,0,39.0,5208608.0,0,0,0,0,0,0,0,0,1,0,1,0
3,0,20.0,5217401.0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,31.0,5208707.0,1,0,0,0,0,0,0,1,1,0,1,0
6,0,28.0,5209937.0,0,1,0,0,0,1,0,0,1,0,0,0
7,0,22.0,5222054.0,1,0,0,0,0,1,0,1,1,0,0,0


In [9]:
# Transformando número da idade e da cidade em inteiros.

df['municipioIBGE'] = [int(elem) for elem in df['municipioIBGE']]
df['idade'] = [int(elem) for elem in df['idade']]

df.head()

,condicoes,idade,municipioIBGE,resultadoTeste,sexo,paladar,fadiga,olfato,garganta,respirar,febre,tosse,diarreia,coriza,dispineia
2,0,39.0,5208608,0,0,0,0,0,0,0,0,1,0,1,0
3,0,20.0,5217401,0,0,0,0,0,0,0,0,1,0,0,0
4,0,31.0,5208707,1,0,0,0,0,0,0,1,1,0,1,0
6,0,28.0,5209937,0,1,0,0,0,1,0,0,1,0,0,0
7,0,22.0,5222054,1,0,0,0,0,1,0,1,1,0,0,0


In [16]:
# Dividindo os dados por cidades. 
city_list = list(set(df.iloc[:, 2]))
df_city = dict()

for city in city_list:
    df_city[city] = []

for line in tqdm(df.iloc[:,:].values):
    df_city[line[2]].append(line)

print(len(df_city.keys()), len(city_list))

  0%|          | 0/1229059 [00:00<?, ?it/s]

1865 1865


In [19]:
# Colocando em arquivos na pasta data_city
import os, shutil, csv

shutil.rmtree('data_city')
os.mkdir('data_city')
for city in tqdm(city_list):

    with open(f'./data_city/{city}.csv',"w") as my_csv:
        csvWriter = csv.writer(my_csv,delimiter=';')
        csvWriter.writerows(df_city[city])

  0%|          | 0/1865 [00:00<?, ?it/s]

# Mapeamento final das variáveis:

- 0: condicoes
- 1: idade
- 2: municipioIBGE
- 3: resultadoTeste
- 4: sexo
- 5: paladar
- 6: fadiga
- 7: olfato
- 8: garganta
- 9: respirar
- 10: febre
- 11: tosse
- 12: diarreia
- 13: coriza
- 14: dispneia